# Brazilian Marketplace Sales Analysis with SQL

## Project Overview

This project explores a real dataset from the Brazilian e-commerce marketplace [Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce), using SQL for data analysis and business insight generation.  
The goal is to showcase data analysis skills, especially in SQL, and uncover trends related to sales, sellers, delivery logistics, and customer reviews.

### Objectives
- Practice advanced SQL techniques on a real-world dataset
- Extract actionable insights about sales behavior, seller performance, and customer satisfaction
- Translate data into business-relevant conclusions and suggestions

### About the Dataset
- Source: [Kaggle – Olist Brazilian E-Commerce Public Dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)
- Tables include: orders, customers, products, sellers, order reviews, and order items
- Period covered: 2016 to 2018 (with a focus on 2017 and 2018 for comparisons)

### Tools & Skills Used
- SQLite queries executed through Python in a Jupyter Notebook
- SQL concepts applied:
  - Joins and Subqueries
  - Aggregations: `SUM`, `COUNT`, `AVG`
  - Conditional logic: `CASE WHEN`
  - Window functions: `RANK`, `LAG`
  - CTEs (Common Table Expressions)
  - Date manipulation: `strftime`, `julianday`
  - Filtering with `WHERE`, `HAVING`
  - Trend, ranking, and difference calculations

---

## Executive Summary

This analysis explores sales, customer satisfaction, and delivery performance in a Brazilian e-commerce marketplace.  
Using SQL in SQLite and visualized through Jupyter Notebook, the analysis covers orders from 2017 to 2018.

### Key Findings:

- **Revenue grew significantly in 2018**, especially in São Paulo State. Over the sales mix, the revenue growth came from categories like *Health & Beauty*, *Watches & Gifts*, and *Home Utilities*.
- **A quantity-focused sales strategy** (lower average prices, higher volume) outperformed high-price product strategies.
- **12% of 2018 revenue came from new sellers**, reinforcing the value of active seller onboarding.
- **Delivery delays were the main cause of bad customer reviews**, especially in states with logistical challenges.
- **The 17-day average delivery time created a one-month lag between delivery issues and customer reviews.

### Suggested Business Actions:

1. **Improve logistics operations in key delay-heavy states**, especially CE, SE, and MA.
2. **Refine estimated delivery dates** shown to customers to reduce expectations gaps.
3. **Onboard new sellers in fast-selling categories** to increase revenue diversity.
4. **Monitor logistics partners’ handoff-to-delivery times**, especially during peak months.
5. **Double down on São Paulo**, as it's the highest-performing region in both revenue and growth.


In [1]:
#Libraries to run the analysis in jupyter lab

import os
import sqlite3
import pandas as pd

#forces the code to look to the right files
os.chdir(r'C:\Users\Vezpa\Mktplace')

# Connect to SQLite archive
conn = sqlite3.connect('olist.sqlite')



In [2]:
#Table overview

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


                                 name
0   product_category_name_translation
1                             sellers
2                           customers
3                         geolocation
4                         order_items
5                      order_payments
6                       order_reviews
7                              orders
8                            products
9                     leads_qualified
10                       leads_closed


# Sales Analysis

The first step of this analysis is to understand the sales behavior of delivered orders, excluding cancellations and orders still in transit that may be canceled.

In [3]:
sales_through_time = """

    SELECT
    *
    ,ROUND(sales - LAG(sales) OVER (ORDER BY year, month ASC),1) difference_MoM
    ,ROUND(((sales - LAG(sales) OVER (ORDER BY year, month ASC)) / LAG(sales) OVER (ORDER BY year, month ASC))*100,1) pct_difference_MoM    
    ,ROUND(sales - LAG(sales,12,null) OVER (ORDER BY year, month ASC),1) difference_YoY
    ,ROUND(((sales - LAG(sales,12,null) OVER (ORDER BY year, month ASC)) / LAG(sales,12,null) OVER (ORDER BY year, month ASC))*100,1) pct_difference_YoY

    FROM (SELECT
            SUBSTR(order_purchase_timestamp,1,4) year
            ,SUBSTR(order_purchase_timestamp,6,2) month
            ,ROUND(SUM(OI.price),1) sales

            FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        

            WHERE O.order_status = 'delivered' AND SUBSTR(order_purchase_timestamp,1,4)<> "2016"
            

        GROUP BY 2,1
        
        ORDER BY 1,2 ASC)
"""

pd.read_sql_query(sales_through_time, conn)



,year,month,sales,difference_MoM,pct_difference_MoM,difference_YoY,pct_difference_YoY
0,2017,01,111798.4,NaN,NaN,NaN,NaN
1,2017,02,234223.4,122425.0,109.5,NaN,NaN
2,2017,03,359198.8,124975.4,53.4,NaN,NaN
3,2017,04,340669.7,-18529.1,-5.2,NaN,NaN
4,2017,05,489338.3,148668.6,43.6,NaN,NaN
5,2017,06,421923.4,-67414.9,-13.8,NaN,NaN
6,2017,07,481604.5,59681.1,14.1,NaN,NaN
7,2017,08,554699.7,73095.2,15.2,NaN,NaN
8,2017,09,607399.7,52700.0,9.5,NaN,NaN
9,2017,10,648247.7,40848.0,6.7,NaN,NaN


## Key Insights

- In every month of 2018, sales were higher than in the same month of the previous year.  
- In 2017, there was a strong month-over-month growth, indicating a significant acceleration during the first year of operations.  
    - At least four months in 2017 saw sales increases of 40% or more.  
- A potential seasonality effect was observed in June, where both 2017 and 2018 experienced a 12–13% drop in sales compared to the previous month.

Based on these findings, we’ll now dive deeper into the key dimensions driving sales: **Sellers**, **Geography**, **Products**, and **Order Cancellations**.

# Geographic Sales Analysis

In [4]:
sales_by_customer_state = """

WITH sales_2018 as (SELECT

        C.customer_state
        ,SUM(OI.price) value
        ,SUM(OI.price) /
            (
                SELECT SUM(OI.price)        
                FROM orders O
                LEFT JOIN order_items OI ON OI.order_id = O.order_id
                WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2018"
            ) sales_percentage

        FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN customers C ON C.customer_id = O.customer_id

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2018"

        GROUP BY 1

        ORDER BY SUM(OI.price) DESC) 



SELECT * 
    ,sales_pct_2018 - sales_pct_2017 pct_var_YoY
    ,SUM(sales_pct_2017) OVER (ORDER BY value_2018 DESC) cummulative_pct_2017
    ,SUM(sales_pct_2018) OVER (ORDER BY value_2018 DESC) cummulative_pct_2018
    
   FROM (SELECT

        C.customer_state
        ,ROUND(S18.value,1) value_2018
        ,ROUND(SUM(OI.price),1) value_2017
        ,ROUND(S18.value - SUM(OI.price),1) sales_var_YoY
        ,ROUND(S18.sales_percentage*100,1) sales_pct_2018
        ,ROUND((SUM(OI.price) /
            (
                SELECT SUM(OI.price)        
                FROM orders O
                LEFT JOIN order_items OI ON OI.order_id = O.order_id
                WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2017"
            ))*100,1) sales_pct_2017

        FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN customers C ON C.customer_id = O.customer_id
        LEFT JOIN sales_2018 S18 ON C.customer_state = S18.customer_state

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2017"

        GROUP BY 1

        ORDER BY SUM(OI.price) DESC) state_sales

    ;
"""

pd.read_sql_query(sales_by_customer_state, conn)


,customer_state,value_2018,value_2017,sales_var_YoY,sales_pct_2018,sales_pct_2017,pct_var_YoY,cummulative_pct_2017,cummulative_pct_2018
0,SP,2919328.2,2136259.0,783069.2,40.4,35.8,4.6,35.8,40.4
1,RJ,874934.1,876499.7,-1565.6,12.1,14.7,-2.6,50.5,52.5
2,MG,840172.2,708034.7,132137.5,11.6,11.9,-0.3,62.4,64.1
3,PR,376623.2,287424.8,89198.4,5.2,4.8,0.4,67.2,69.3
4,RS,375581.1,350792.8,24788.4,5.2,5.9,-0.7,73.1,74.5
5,SC,281524.7,223249.3,58275.4,3.9,3.7,0.2,76.8,78.4
6,BA,265885.8,227453.4,38432.4,3.7,3.8,-0.1,80.6,82.1
7,DF,164284.9,131169.7,33115.2,2.3,2.2,0.1,82.8,84.4
8,ES,152031.1,115752.6,36278.5,2.1,1.9,0.2,84.7,86.5
9,GO,148131.2,133801.0,14330.2,2.1,2.2,-0.1,86.9,88.6


## States Ranked by Revenue Growth from 2017 to 2018

In [5]:
sales_by_state = """

WITH sales_2018 as (SELECT

        C.customer_state
        ,SUM(OI.price) value

        FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN customers C ON C.customer_id = O.customer_id

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2018"

        GROUP BY 1

        ORDER BY SUM(OI.price) DESC) 



SELECT 
       RANK() OVER(ORDER BY S18.value - SUM(OI.price) DESC) as Revenue_Variation_Rank
        ,C.customer_state
        ,ROUND(S18.value,1) value_2018
        ,ROUND(SUM(OI.price),1) value_2017
        ,ROUND(S18.value - SUM(OI.price),1) sales_diff_2018_to_2017
        ,ROUND(((S18.value - SUM(OI.price))/SUM(OI.price))*100,1) State_YoY

        FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN customers C ON C.customer_id = O.customer_id
        LEFT JOIN sales_2018 S18 ON C.customer_state = S18.customer_state

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= "2017"

        GROUP BY 2

        ORDER BY S18.value - SUM(OI.price) DESC

    ;
"""

pd.read_sql_query(sales_by_state, conn)


,Revenue_Variation_Rank,customer_state,value_2018,value_2017,sales_diff_2018_to_2017,State_YoY
0,1,SP,2919328.2,2136259.0,783069.2,36.7
1,2,MG,840172.2,708034.7,132137.5,18.7
2,3,PR,376623.2,287424.8,89198.4,31.0
3,4,SC,281524.7,223249.3,58275.4,26.1
4,5,BA,265885.8,227453.4,38432.4,16.9
5,6,ES,152031.1,115752.6,36278.5,31.3
6,7,DF,164284.9,131169.7,33115.2,25.2
7,8,RS,375581.1,350792.8,24788.4,7.1
8,9,PE,134193.9,116446.0,17747.9,15.2
9,10,GO,148131.2,133801.0,14330.2,10.7


## City-Level Revenue Rankings Within the Top States

In [6]:
sales_by_customer_state_and_city = """
WITH city_sales_2017 AS (
SELECT
        CS.customer_city
        ,SUM(OI.price) value     

        FROM orders O

        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN customers CS ON CS.customer_id = O.customer_id

        WHERE O.order_status = 'delivered' 
            AND CS.customer_state IN ('SP','MG','PR','SC','BA') 
            AND SUBSTR(O.order_purchase_timestamp,1,4)= "2017"

        GROUP BY 1

        ORDER BY SUM(OI.price) DESC)



SELECT * 

        FROM (SELECT

                CS.customer_city
                ,CS.customer_state
                ,SUM(OI.price) value_2018
                ,C17.value value_2017
                ,SUM(OI.price) - C17.value value_diff_YoY
                ,RANK() OVER(PARTITION BY CS.customer_state ORDER BY SUM(OI.price) - C17.value DESC) AS city_ranking_inside_state
                ,RANK() OVER(ORDER BY SUM(OI.price) - C17.value DESC) AS general_ranking

                FROM orders O

                LEFT JOIN order_items OI ON OI.order_id = O.order_id
                LEFT JOIN customers CS ON CS.customer_id = O.customer_id
                LEFT JOIN city_sales_2017 C17 ON CS.customer_city = C17.customer_city

                WHERE O.order_status = 'delivered' 
                    AND CS.customer_state IN ('SP','MG','PR','SC','BA')
                    AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018'

                GROUP BY 1,2

                ORDER BY general_ranking ASC) initial_ranking
                
            WHERE general_ranking <= 15
    
    
    ;
"""


pd.read_sql_query(sales_by_customer_state_and_city, conn)

,customer_city,customer_state,value_2018,value_2017,value_diff_YoY,city_ranking_inside_state,general_ranking
0,sao paulo,SP,1103641.29,752425.64,351215.65,1,1
1,belo horizonte,MG,199179.91,145665.88,53514.03,1,2
2,curitiba,PR,122566.77,83014.88,39551.89,1,3
3,santos,SP,64087.29,33808.78,30278.51,2,4
4,guarulhos,SP,81484.69,57443.60,24041.09,3,5
5,campinas,SP,103429.74,80776.55,22653.19,4,6
6,jundiai,SP,47768.89,32629.15,15139.74,5,7
7,sao jose dos campos,SP,45102.54,30305.87,14796.67,6,8
8,santo andre,SP,51151.82,37887.71,13264.11,7,9
9,sao bernardo do campo,SP,57763.57,44647.87,13115.70,8,10


- São Paulo is the only state that increased its share of national sales by more than 1%.  
- SP, MG, PR, SC, and BA were the states with the largest sales growth in 2018 (up to August), compared to the full year of 2017.  
- The sales increase was highly concentrated in cities within São Paulo state: 11 of the top 15 cities were in SP, with only the capital cities of other states appearing in the ranking.

# Sales Mix Analysis

We begin by analyzing the revenue, quantity of items sold, and average price by product category to better understand the composition of the sales mix.

## Top Revenue Categories of 2018

In [7]:
top_15_revenues_of_2018 = """WITH ly_sales AS (
    SELECT 
        P.product_category_name,
        SUM(OI.price) AS revenue,
        RANK() OVER (ORDER BY COUNT(OI.product_id) DESC) AS qty_rank_2017,
        RANK() OVER (ORDER BY SUM(OI.price)*1.0 / COUNT(OI.product_id) DESC) AS avg_price_rank_2017
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id = P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2017'
    GROUP BY P.product_category_name
),

sales_2018 AS (
    SELECT     
        P.product_category_name,
        ROUND(SUM(OI.price) * 100.0 / (
            SELECT SUM(OI.price)
            FROM order_items OI
            LEFT JOIN orders O ON OI.order_id = O.order_id
            WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
        ), 2) AS revenue_share_2018,
        SUM(OI.price) AS revenue_2018,
        ly.revenue AS revenue_2017,
        SUM(OI.price) - ly.revenue AS dif_revenue,
        RANK() OVER (ORDER BY COUNT(OI.product_id) DESC) AS qty_rank_2018,
        ly.qty_rank_2017,
        RANK() OVER (ORDER BY SUM(OI.price)*1.0 / COUNT(OI.product_id) DESC) AS avg_price_rank_2018,
        ly.avg_price_rank_2017,
        RANK() OVER (ORDER BY SUM(OI.price) DESC) AS sales_ranked_2018
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id = P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    LEFT JOIN ly_sales ly ON ly.product_category_name = P.product_category_name
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
    GROUP BY P.product_category_name
    HAVING SUM(OI.price) > 10000
)

SELECT *,
    SUM(revenue_share_2018) OVER (ORDER BY revenue_share_2018 DESC) AS cummulative_revenue_2018
FROM sales_2018
WHERE sales_ranked_2018 <= 15;
"""

pd.read_sql_query(top_15_revenues_of_2018, conn)

,product_category_name,revenue_share_2018,revenue_2018,revenue_2017,dif_revenue,qty_rank_2018,qty_rank_2017,avg_price_rank_2018,avg_price_rank_2017,sales_ranked_2018,cummulative_revenue_2018
0,beleza_saude,10.47,755724.50,473833.00,281891.50,1,4,25,28,1,10.47
1,relogios_presentes,9.53,687855.20,475610.71,212244.49,7,10,9,11,2,20.00
2,cama_mesa_banho,7.38,532358.85,490596.92,41761.93,2,1,40,48,3,27.38
3,esporte_lazer,7.16,517166.26,435674.14,81492.12,4,3,28,40,4,34.54
4,informatica_acessorios,6.88,496269.30,391786.29,104483.01,3,5,35,30,5,41.42
5,utilidades_domesticas,5.43,391823.46,222518.16,169305.30,6,6,38,57,6,46.85
6,moveis_decoracao,5.29,381649.57,324587.60,57061.97,5,2,39,56,7,52.14
7,automotivo,4.76,343288.30,234550.09,108738.21,8,13,22,21,8,56.90
8,bebes,3.47,250615.79,148434.88,102180.91,12,14,19,36,9,60.37
9,cool_stuff,3.16,227743.70,381414.30,-153670.60,16,9,15,17,10,63.53


A significant portion of revenue is concentrated in a few product categories.

- In 2018, the top 10 best-selling categories accounted for 63% of total sales, and the top 15 made up 75%.

Selling a large volume of items is more impactful than maintaining a high average price per item.

- 13 out of the 15 highest-revenue categories also appear in the top 15 for items sold, while only 2 appear in the top 15 for average price.
- This pattern was also observed in the 2017 sales data.

## Average Sales Price by Category

In [8]:
avg_price_items ="""

WITH ly_sales AS (

SELECT 
    P.product_category_name
    ,SUM(OI.price) / COUNT(OI.product_id) avg_revenue
    ,RANK() OVER(ORDER BY SUM(OI.price) DESC) revenue_rank_2017
    ,RANK() OVER(ORDER BY COUNT(OI.product_id) DESC) qty_rank_2017
    
    
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id= P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2017'
    
    GROUP BY 1
    
    )
    
    SELECT * 
    FROM
        (SELECT
            RANK() OVER(ORDER BY SUM(OI.price) / COUNT(OI.product_id) DESC) rank_avg_price
            ,P.product_category_name
            ,ROUND(SUM(OI.price) / COUNT(OI.product_id),2) avg_price_2018
            ,ROUND(ly.avg_revenue,2) avg_price_2017
            ,ROUND((SUM(OI.price) / COUNT(OI.product_id)),2) - ly.avg_revenue dif_avg_price
            ,RANK() OVER(ORDER BY SUM(OI.price) DESC) revenue_rank_2018
            ,ly.revenue_rank_2017
            ,RANK() OVER(ORDER BY COUNT(OI.product_id) DESC) qty_rank_2018
            ,ly.qty_rank_2017

        FROM products P
        LEFT JOIN order_items OI ON OI.product_id= P.product_id
        LEFT JOIN orders O ON OI.order_id = O.order_id
        LEFT JOIN ly_sales ly ON ly.product_category_name = P.product_category_name

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018'

        GROUP BY 2) general_table
        
    WHERE general_table.rank_avg_price <=15
    
    ORDER BY general_table.rank_avg_price ASC
        
    """

pd.read_sql_query(avg_price_items, conn)


,rank_avg_price,product_category_name,avg_price_2018,avg_price_2017,dif_avg_price,revenue_rank_2018,revenue_rank_2017,qty_rank_2018,qty_rank_2017
0,1,pcs,866.71,1230.56,-363.847480,26,13.0,50,35.0
1,2,portateis_casa_forno_e_cafe,655.07,244.93,410.140000,30,64.0,51,70.0
2,3,eletrodomesticos_2,587.18,253.63,333.552410,21,33.0,42,42.0
3,4,agro_industria_e_comercio,285.43,478.35,-192.917541,31,28.0,43,45.0
4,5,portateis_cozinha_e_preparadores_de_alimentos,280.97,NaN,NaN,56,NaN,67,NaN
5,6,telefonia_fixa,270.05,184.03,86.017067,38,29.0,47,32.0
6,7,instrumentos_musicais,269.28,304.22,-34.938837,19,23.0,26,25.0
7,8,eletroportateis,246.88,312.39,-65.510903,22,18.0,28,23.0
8,9,construcao_ferramentas_seguranca,216.27,196.17,20.097500,35,46.0,45,51.0
9,10,pc_gamer,196.75,129.99,66.760000,67,68.0,70,68.0


- Only one of the highest average price categories — `relogios_presentes` — appears in the top revenue list, reinforcing the conclusion that the main share of revenue is driven by quantity sold rather than high item prices.

## Categories Ranked by Year-over-Year Revenue Growth


In [9]:
relevant_revenue_items ="""

WITH ly_sales AS (

SELECT 
    P.product_category_name
    ,SUM(OI.price) revenue
    ,ROUND(SUM(OI.price) / 
        (SELECT SUM(t17.price) 
            FROM order_items t17
            LEFT JOIN orders O ON OI.order_id = O.order_id 
            LEFT JOIN customers CS ON CS.customer_id = O.customer_id
            WHERE O.order_status = 'delivered' 
            AND SUBSTR(O.order_purchase_timestamp,1,4)= '2017')*100
                ,2) share_2017

    
    
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id= P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2017'
    
    GROUP BY 1
    
    )
    
    
    SELECT     
        P.product_category_name
        ,SUM(OI.price) revenue_2018
        ,ly.revenue revenue_2017
        ,SUM(OI.price) - ly.revenue dif_revenue
        ,ROUND(SUM(OI.price) / 
            (SELECT SUM(t17.price) 
                FROM order_items t17
                LEFT JOIN orders O ON OI.order_id = O.order_id 
                LEFT JOIN customers CS ON CS.customer_id = O.customer_id
                WHERE O.order_status = 'delivered' 
                AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018')*100
                    ,2) share_2018
          ,ly.share_2017

    
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id= P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    LEFT JOIN ly_sales ly ON ly.product_category_name = P.product_category_name
    
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018'
    
    GROUP BY 1
    
    HAVING SUM(OI.price) - ly.revenue  > 10000 OR SUM(OI.price) - ly.revenue  < -10000
    
    ORDER BY 4 DESC
    
    """

pd.read_sql_query(relevant_revenue_items, conn)



,product_category_name,revenue_2018,revenue_2017,dif_revenue,share_2018,share_2017
0,beleza_saude,755724.50,473833.00,281891.50,5.56,3.49
1,relogios_presentes,687855.20,475610.71,212244.49,5.06,3.50
2,utilidades_domesticas,391823.46,222518.16,169305.30,2.88,1.64
3,automotivo,343288.30,234550.09,108738.21,2.53,1.73
4,informatica_acessorios,496269.30,391786.29,104483.01,3.65,2.88
5,construcao_ferramentas_construcao,122886.61,19221.88,103664.73,0.90,0.14
6,bebes,250615.79,148434.88,102180.91,1.84,1.09
7,esporte_lazer,517166.26,435674.14,81492.12,3.81,3.21
8,eletrodomesticos_2,86902.86,21051.09,65851.77,0.64,0.15
9,moveis_decoracao,381649.57,324587.60,57061.97,2.81,2.39


- The categories `beleza_saude` and `relogios_presentes` showed the highest revenue growth within the overall sales mix.

## Average sales price of the top revenue categories

In [10]:
sales_price_items ="""

WITH ly_sales AS (

SELECT 
    P.product_category_name
    ,AVG(OI.price) avg_price_itens
    
    
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id= P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2017'
    AND P.product_category_name IN ('beleza_saude','utilidades_domesticas','relogios_presentes','informatica_acessorios','bebes','construcao_ferramentas_construcao','automotivo','cama_mesa_banho','esporte_lazer','pet_shop')
    
    GROUP BY 1
    
    )
    
    
    SELECT     
        P.product_category_name
        ,AVG(OI.price) avg_price_itens_2018
        ,ly.avg_price_itens avg_price_itens_2017
        ,AVG(OI.price) - ly.avg_price_itens dif_price_avg
        ,ROUND(((AVG(OI.price) - ly.avg_price_itens) / ly.avg_price_itens)*100,2)pct_price_var
    
    FROM products P
    LEFT JOIN order_items OI ON OI.product_id= P.product_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    LEFT JOIN ly_sales ly ON ly.product_category_name = P.product_category_name

    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018' 
    AND P.product_category_name IN ('beleza_saude','utilidades_domesticas','relogios_presentes','informatica_acessorios','bebes','construcao_ferramentas_construcao','automotivo','cama_mesa_banho','esporte_lazer','pet_shop')
        
    GROUP BY 1
    ;
    
    """

pd.read_sql_query(sales_price_items, conn)


,product_category_name,avg_price_itens_2018,avg_price_itens_2017,dif_price_avg,pct_price_var
0,automotivo,133.057481,151.127635,-18.070155,-11.96
1,bebes,144.780930,119.705548,25.075382,20.95
2,beleza_saude,129.382726,132.318626,-2.935901,-2.22
3,cama_mesa_banho,91.628029,95.539809,-3.911780,-4.09
4,construcao_ferramentas_construcao,156.145629,149.006822,7.138807,4.79
5,esporte_lazer,116.583918,109.520900,7.063018,6.45
6,informatica_acessorios,107.371116,130.161558,-22.790442,-17.51
7,pet_shop,110.689040,108.729096,1.959945,1.80
8,relogios_presentes,189.596251,213.565653,-23.969402,-11.22
9,utilidades_domesticas,98.720952,79.075394,19.645558,24.84


- 90% of the top-revenue categories have average prices between R$91 and R$156. Only `relogios_presentes` exceeds this range.  
This reinforces that total revenue is primarily driven by high sales volume rather than high average item price.

# Revenue by Sellers

In [11]:
revenue_by_sellers ="""

WITH ly_sales AS (

SELECT 
    S.seller_id
    ,SUM(OI.price) revenue
    ,SUM(OI.price) / (SELECT 
        SUM(PP.price)
        FROM order_items PP LEFT JOIN orders X ON PP.order_id = X.order_id 
        WHERE X.order_status = 'delivered' AND SUBSTR(X.order_purchase_timestamp,1,4)= '2017') share_2017
    
    FROM sellers S
    LEFT JOIN order_items OI ON OI.seller_id= S.seller_id
    LEFT JOIN orders O ON OI.order_id = O.order_id
    
    WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2017'
    
    GROUP BY 1
    
    )
    
SELECT     
            S.seller_id
            ,SUM(OI.price) revenue_2018
            ,ly.revenue revenue_2017
            ,SUM(OI.price) - ly.revenue dif_revenue
            ,ROUND(SUM(OI.price) * 100.0 / (
            SELECT SUM(OI.price)
            FROM order_items OI
            LEFT JOIN orders O ON OI.order_id = O.order_id
            WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
            ), 2)share_2018
    
            ,ROUND((ly.share_2017*100),2) share_2017

        FROM sellers S
        LEFT JOIN order_items OI ON OI.seller_id= S.seller_id
        LEFT JOIN orders O ON OI.order_id = O.order_id
        LEFT JOIN ly_sales ly ON ly.seller_id = S.seller_id

        WHERE O.order_status = 'delivered' AND SUBSTR(O.order_purchase_timestamp,1,4)= '2018'

        GROUP BY 1
        
        HAVING SUM(OI.price) > 10000
    
        ORDER BY 2 DESC
    
    """

pd.read_sql_query(revenue_by_sellers, conn)

,seller_id,revenue_2018,revenue_2017,dif_revenue,share_2018,share_2017
0,4869f7a5dfa277a7dca6462dcf3b52b2,136164.90,90823.03,45341.87,1.89,1.52
1,955fee9216a65b617aa5c0531780ce60,114280.86,17555.85,96725.01,1.58,0.29
2,7d13fca15225358621be4086e1eb0964,112436.18,NaN,NaN,1.56,NaN
3,1025f0e2d44d7041d6cf58b6550e0bfa,104973.71,33234.85,71738.86,1.45,0.56
4,fa1c13f2614d7b5c4749cbc52fecda94,93687.82,97229.32,-3541.50,1.30,1.63
...,...,...,...,...,...,...
144,080199a181c46c657dc5aa235411be3b,10300.44,4203.76,6096.68,0.14,0.07
145,33a6f4b1e7cdc205511e76ba1b6e0186,10295.01,NaN,NaN,0.14,NaN
146,c510bc1718f0f2961eaa42a23330681a,10141.00,NaN,NaN,0.14,NaN
147,54a1852d1b8f10312c55e906355666ee,10102.73,6953.73,3149.00,0.14,0.12


The marketplace is not dependent on any single seller.

- Sales revenue is highly distributed across the seller base, with the top seller accounting for only 1.9% of total revenue.

## Share of Revenue from New Sellers (Started in 2018)

In [12]:
revenue_items ="""

SELECT     
    S.seller_id
    ,SUM(OI.price) AS revenue_2018
    ,ROUND(
        (SUM(OI.price) * 100.0) /
        (
            SELECT SUM(PP.price)
            FROM order_items PP
            LEFT JOIN orders O ON PP.order_id = O.order_id 
            WHERE O.order_status = 'delivered' 
              AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
        ), 
    2) AS share_2018
    ,SUM(
    ROUND(
        (SUM(OI.price) * 100.0) /
        (
            SELECT SUM(PP.price)
            FROM order_items PP
            LEFT JOIN orders O ON PP.order_id = O.order_id 
            WHERE O.order_status = 'delivered' 
              AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
        ), 
    2)) OVER (ORDER BY SUM(OI.price) DESC) Cummulative_share

FROM sellers S
LEFT JOIN order_items OI ON OI.seller_id = S.seller_id
LEFT JOIN orders O ON OI.order_id = O.order_id 

WHERE O.order_status = 'delivered' 
  AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2018'
  AND S.seller_id NOT IN (
      SELECT seller_id
      FROM (
          SELECT 
              S.seller_id,
              SUM(OI.price) AS revenue_2017
          FROM sellers S
          LEFT JOIN order_items OI ON OI.seller_id = S.seller_id
          LEFT JOIN orders O ON OI.order_id = O.order_id
          WHERE O.order_status = 'delivered' 
            AND SUBSTR(O.order_purchase_timestamp, 1, 4) = '2017'
          GROUP BY S.seller_id
          HAVING SUM(OI.price) > 0.01
      )
  )

GROUP BY S.seller_id
HAVING SUM(OI.price) > 10000
ORDER BY revenue_2018 DESC;

    
    """

pd.read_sql_query(revenue_items, conn)


,seller_id,revenue_2018,share_2018,Cummulative_share
0,7d13fca15225358621be4086e1eb0964,112436.18,1.56,1.56
1,de722cd6dad950a92b7d4f82673f8833,55126.30,0.76,2.32
2,ceaec5548eefc6e23e6607c5435102e7,51478.18,0.71,3.03
3,e882b2a25a10b9c057cc49695f222c19,51057.54,0.71,3.74
4,966cb4760537b1404caedd472cc610a5,45928.26,0.64,4.38
5,ba90964cff9b9e0e6f32b23b82465f7b,44212.50,0.61,4.99
6,b839e41795b7f3ad94cc2014a52f6796,42829.00,0.59,5.58
7,2eb70248d66e0e3ef83659f71b244378,38990.72,0.54,6.12
8,c70c1b0d8ca86052f45a432a38b73958,36289.15,0.50,6.62
9,06a2c3af7b3aee5d69171b0e14f0ee87,36097.98,0.50,7.12


- New sellers contributed nearly 13% of total revenue in 2018 — a significant share, even though the majority of sales were still driven by existing sellers.

## Seller Count by Cancellation Rate

In [13]:
CANCELLED_SALES_SELLER = """

WITH cancelled_sales AS (

    SELECT
        S.seller_id
        ,SUM(OI.price) cancelled_value

        FROM orders O
        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN sellers S ON S.seller_id = OI.seller_id

        WHERE O.order_status = 'canceled'

        GROUP BY 1

        ORDER BY SUM(OI.price) DESC)

    SELECT
    cancelations_rate,
    COUNT(*) qtd_of_sellers,
    SUM(value) revenue,
    SUM(cancelled_value) lost_revenue
    
    
    FROM (SELECT
        S.seller_id
        ,RANK() OVER(ORDER BY (CS.cancelled_value / SUM(OI.price))  DESC) share_cancelations_rate
        ,ROUND((CS.cancelled_value / SUM(OI.price))*100,0) cancelations_rate
        ,SUM(OI.price) value
        ,CS.cancelled_value
        

        FROM orders O
        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN sellers S ON S.seller_id = OI.seller_id
        LEFT JOIN cancelled_sales CS on CS.seller_id = S.seller_id

        GROUP BY 1) CANCELATION_RATE
        
        GROUP BY 1
        ORDER BY 1 DESC
        ;
    

    """

pd.read_sql_query(CANCELLED_SALES_SELLER,conn)

,cancelations_rate,qtd_of_sellers,revenue,lost_revenue
0,100.0,39,6996.09,6996.09
1,89.0,1,727.90,650.00
2,88.0,1,1020.00,900.00
3,80.0,1,694.77,557.00
4,77.0,1,11961.83,9199.80
5,74.0,1,696.00,518.00
6,72.0,1,1304.67,933.89
7,71.0,1,244.90,175.00
8,66.0,1,105.15,69.80
9,65.0,1,1680.10,1095.80


Cancellations do not represent a major issue in terms of overall revenue or seller performance.

- 39 sellers had a 100% cancellation rate, meaning that the only order they fulfilled was ultimately canceled.  
- This represents a very small portion of the marketplace, especially when compared to the total seller base of nearly 3,000.  
- The total revenue associated with canceled orders is also minimal when compared to overall marketplace revenue.

# Customer Reviews Analysis

Next, we’ll analyze how customers are rating their experiences with both deliveries and products.

In [14]:
customer_review = """

    SELECT
        strftime('%Y-%m', review_creation_date) AS month_year
       ,AVG(review_score) average_review
       ,COUNT(*) qty_reviews
        

        FROM order_reviews  
        group by 1
        
"""
pd.read_sql_query(customer_review, conn)

,month_year,average_review,qty_reviews
0,2016-10,4.055866,179
1,2016-11,3.188119,101
2,2016-12,2.355556,45
3,2017-01,4.338912,239
4,2017-02,4.280962,1413
5,2017-03,4.033051,2481
6,2017-04,4.036983,2055
7,2017-05,4.100539,3710
8,2017-06,4.127616,3440
9,2017-07,4.183271,3503


There were spikes in review volume and noticeable drops in average review scores during December 2017, March 2018, and April 2018.

# Customer reviews by state

In [15]:
customer_review = """

    SELECT
        C.customer_state
       ,AVG(R.review_score) average_review
       ,COUNT(R.review_id) qty_reviews
        

        FROM order_reviews R 
        LEFT JOIN orders O ON O.order_id = R.order_id
        LEFT JOIN customers C ON C.customer_id = O.customer_id
        GROUP BY 1
        ORDER BY 2 DESC
        
"""
pd.read_sql_query(customer_review, conn)


,customer_state,average_review,qty_reviews
0,AP,4.194030,67
1,AM,4.183673,147
2,PR,4.180032,5038
3,SP,4.173951,41690
4,MG,4.136172,11625
5,RS,4.133321,5483
6,MS,4.118785,724
7,RN,4.105809,482
8,MT,4.102990,903
9,TO,4.096774,279


- Nine states have an average customer review score below 4.0.  
- One of them — Rio de Janeiro (RJ) — is among the top five states in total revenue.

## Reviews by Sellers

In [16]:
x ="""
SELECT *
,RANK() OVER( ORDER BY avg_review DESC) reviews_rank

FROM (select

S.seller_id
,ROUND(AVG(R.review_score),2) avg_review
,COUNT(R.review_score) qty_review
,RANK() OVER(ORDER BY COUNT(R.review_score) DESC) rank_qty_review



FROM orders O
LEFT JOIN order_reviews R ON R.order_id = O.order_id
LEFT JOIN order_items OI ON OI.order_id = O.order_id
LEFT JOIN sellers S ON S.seller_id = OI.seller_id

GROUP BY 1

ORDER BY 3 DESC)

WHERE rank_qty_review <=50

"""        
pd.read_sql_query(x, conn)


,seller_id,avg_review,qty_review,rank_qty_review,reviews_rank
0,7299e27ed73d2ad986de7f7c77d919fa,4.42,361,47,1
1,fa1c13f2614d7b5c4749cbc52fecda94,4.34,582,22,2
2,9f505651f4a6abe901a56cdc21508025,4.28,427,37,3
3,7a67c85e85bb2ce8582c35f2203ad736,4.23,1166,10,4
4,8581055ce74af1daba164fdbd55a40de,4.23,442,32,4
5,dbc22125167c298ef99da25668e1011f,4.23,429,34,4
6,620c87c171fb2a6dd6e8bb4dec959fc6,4.22,790,15,7
7,f8db351d8c4c4c22c6835c19a46f01b0,4.22,722,19,7
8,a1043bafd471dff536d0c462352beb48,4.19,767,16,9
9,128639473a139ac0f3e5f5ade55873a5,4.18,554,25,10


We will analyze the customer review and sales history of the lowest-rated seller to understand the impact of negative reviews on sales performance.

In [17]:

sales_through_time = """

WITH reviews AS(

        SELECT
        strftime('%Y-%m', review_creation_date) AS month_year
       ,AVG(review_score) average_review  

        
        FROM orders O
        LEFT JOIN order_reviews R ON R.order_id = O.order_id
        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN sellers S ON S.seller_id = OI.seller_id  
        
        WHERE S.seller_id  = '7c67e1448b00f6e969d365cea6b010ab' AND strftime('%Y-%m', review_creation_date) IS NOT null
        GROUP BY 1)



    SELECT
    *
    ,ROUND(sales - LAG(sales) OVER (ORDER BY month_year ASC),1) difference_MoM
    ,ROUND(((sales - LAG(sales) OVER (ORDER BY month_year ASC)) / LAG(sales) OVER (ORDER BY month_year ASC))*100,1) pct_difference_MoM    
    ,ROUND(sales - LAG(sales,12,null) OVER (ORDER BY month_year ASC),1) difference_YoY
    ,ROUND(((sales - LAG(sales,12,null) OVER (ORDER BY month_year ASC)) / LAG(sales,12,null) OVER (ORDER BY month_year ASC))*100,1) pct_difference_YoY

    FROM (SELECT
            strftime('%Y-%m', order_purchase_timestamp) AS month_year
            ,ROUND(SUM(OI.price),1) sales
            ,rev.average_review
            ,COUNT(DISTINCT O.order_id) orders_qty
            ,COUNT(OI.order_item_id) items_sold
            ,ROUND(AVG(OI.price),2) average_price

            FROM orders O

            LEFT JOIN order_items OI ON OI.order_id = O.order_id
            LEFT JOIN sellers S ON S.seller_id = OI.seller_id
            LEFT JOIN reviews rev ON strftime('%Y-%m', order_purchase_timestamp) = rev.month_year


            WHERE O.order_status = 'delivered' AND SUBSTR(order_purchase_timestamp,1,4)<> "2016" AND S.seller_id = '7c67e1448b00f6e969d365cea6b010ab'


            GROUP BY 1

            ORDER BY 1 ASC)
"""

pd.read_sql_query(sales_through_time, conn)

,month_year,sales,average_review,orders_qty,items_sold,average_price,difference_MoM,pct_difference_MoM,difference_YoY,pct_difference_YoY
0,2017-01,1365.3,NaN,8,10,136.53,NaN,NaN,NaN,NaN
1,2017-02,4900.0,3.900000,32,37,132.43,3534.7,258.9,NaN,NaN
2,2017-03,6896.9,2.925000,41,55,125.40,1996.9,40.8,NaN,NaN
3,2017-04,4023.4,3.538462,23,26,154.75,-2873.5,-41.7,NaN,NaN
4,2017-05,7087.7,3.833333,36,49,144.65,3064.3,76.2,NaN,NaN
5,2017-06,6703.6,4.225806,40,43,155.90,-384.1,-5.4,NaN,NaN
6,2017-07,15661.2,3.686567,91,108,145.01,8957.6,133.6,NaN,NaN
7,2017-08,8614.8,3.824074,44,57,151.14,-7046.4,-45.0,NaN,NaN
8,2017-09,6433.3,4.057143,30,52,123.72,-2181.5,-25.3,NaN,NaN
9,2017-10,14875.5,3.315068,58,89,167.14,8442.2,131.2,NaN,NaN


Although this seller ranks as the eighth most reviewed on the platform, it surpassed R$15,000 in monthly revenue in only three months — a relatively low performance compared to other sellers.

In [18]:
customer_review = """

    SELECT
        review_score
        ,review_comment_message
        

        
        FROM orders O
        LEFT JOIN order_reviews R ON R.order_id = O.order_id
        LEFT JOIN order_items OI ON OI.order_id = O.order_id
        LEFT JOIN sellers S ON S.seller_id = OI.seller_id  
        
        WHERE S.seller_id  = '7c67e1448b00f6e969d365cea6b010ab' AND strftime('%Y-%m', review_creation_date) = '2018-03' AND review_score <=3 AND review_comment_message IS NOT NULL
        group by 1,2
        ORDER BY 1

"""
pd.read_sql_query(customer_review, conn)

,review_score,review_comment_message
0,1,- a compra foi efetuada em 30/01\r\n- a emissã...
1,1,"A mesa tem defeitos em sua tabua, e as pernas ..."
2,1,Ainda nao recebi minha mercadoria.
3,1,"Ainda não recebi o produto, já passou do prazo..."
4,1,COMPREI 6 CADEIRAS E SÓ RECEBI 1 E AINDA VEIO ...
5,1,Comprei seis(6) cadeiras e recebi 4(quatro)
6,1,Cumpriu com excelência
7,1,"Estou insatisfeito, pois ainda não recebi meu ..."
8,1,Eu pedi 6 unidades me entregaram 1.
9,1,Nao recebi o produto


Most of the customer comments during March 2018 mention issues such as delivery delays, poor product quality, and incorrect item quantities.

Now, let’s examine seller review scores during the lowest-performing periods: March 2018, April 2018, and December 2017.

## Seller Review Scores During Low-Performing Periods (2017-12, 2018-03, 2018-04)

In [19]:
x ="""

WITH order_reviews_treated AS
    (SELECT * FROM
        (SELECT *,strftime('%Y-%m', review_creation_date) month_year FROM order_reviews) order_review_with_month
        
    WHERE month_year IN ('2018-03','2018-04',"2017-12"))


SELECT *
,RANK() OVER( ORDER BY avg_review DESC) reviews_rank

FROM (select

S.seller_id
,ROUND(AVG(R.review_score),2) avg_review
,COUNT(R.review_score) qty_review
,RANK() OVER(ORDER BY COUNT(R.review_score) DESC) rank_qty_review


FROM orders O
LEFT JOIN order_reviews_treated R ON R.order_id = O.order_id
LEFT JOIN order_items OI ON OI.order_id = O.order_id
LEFT JOIN sellers S ON S.seller_id = OI.seller_id

GROUP BY 1

ORDER BY 3 DESC)

WHERE rank_qty_review <=50



"""        
pd.read_sql_query(x, conn)


,seller_id,avg_review,qty_review,rank_qty_review,reviews_rank
0,9f505651f4a6abe901a56cdc21508025,4.25,96,46,1
1,53e4c6e0f4312d4d2107a8c9cddf45cd,4.20,122,31,2
2,de722cd6dad950a92b7d4f82673f8833,4.15,121,32,3
3,7a67c85e85bb2ce8582c35f2203ad736,4.10,200,14,4
4,a1043bafd471dff536d0c462352beb48,4.09,218,13,5
5,4d6d651bd7684af3fffabd5f08d12e5a,4.07,112,38,6
6,128639473a139ac0f3e5f5ade55873a5,4.06,174,18,7
7,d91fb3b7d041e83b64a00a3edfb37e4f,4.02,131,28,8
8,8581055ce74af1daba164fdbd55a40de,4.00,117,33,9
9,f8db351d8c4c4c22c6835c19a46f01b0,4.00,92,47,9


There are 10 sellers with an average review score below 3.5 during this period.  
Next, we’ll evaluate the impact on the overall customer review average after excluding these sellers from the dataset.

In [20]:

sales_through_time = """
WITH reviews_without_bad_sellers AS
        (
            SELECT
            strftime('%Y-%m', review_creation_date) AS month_year
           ,ROUND(AVG(review_score),2) average_review_without_bad_sellers  


            FROM orders O
            LEFT JOIN order_reviews R ON R.order_id = O.order_id
            LEFT JOIN order_items OI ON OI.order_id = O.order_id
            LEFT JOIN sellers S ON S.seller_id = OI.seller_id  

            WHERE strftime('%Y-%m', review_creation_date) IS NOT null 
                AND S.seller_id NOT IN ('7c67e1448b00f6e969d365cea6b010ab','88460e8ebdecbfecb5f9601833981930','1835b56ce799e6a4dc4eddc053f04066','897060da8b9a21f655304d50fd935913','4c2b230173bb36f9b240f2b8ac11786e','1025f0e2d44d7041d6cf58b6550e0bfa','4a3ca9315b744ce9f8e9374361493884','d2374cbcbb3ca4ab1086534108cc3ab7','8160255418d5aaa7dbdc9f4c64ebda44','e5a3438891c0bfdb9394643f95273d8e')
            GROUP BY 1
        )
        
  SELECT
    strftime('%Y-%m', review_creation_date) AS month_year
   ,RB.average_review_without_bad_sellers
   ,ROUND(AVG(review_score),2) average_review
   ,RB.average_review_without_bad_sellers - ROUND(AVG(review_score),2) diference_score


    FROM orders O
    LEFT JOIN order_reviews R ON R.order_id = O.order_id
    LEFT JOIN order_items OI ON OI.order_id = O.order_id
    LEFT JOIN sellers S ON S.seller_id = OI.seller_id  
    LEFT JOIN reviews_without_bad_sellers RB ON RB.month_year = strftime('%Y-%m', review_creation_date)

    WHERE strftime('%Y-%m', review_creation_date) IS NOT null AND strftime('%Y', review_creation_date) IS NOT '2016'
    GROUP BY 1;

"""

pd.read_sql_query(sales_through_time, conn)

,month_year,average_review_without_bad_sellers,average_review,diference_score
0,2017-01,4.20,4.19,0.01
1,2017-02,4.28,4.27,0.01
2,2017-03,4.09,4.01,0.08
3,2017-04,4.01,3.97,0.04
4,2017-05,4.10,4.07,0.03
5,2017-06,4.13,4.09,0.04
6,2017-07,4.17,4.12,0.05
7,2017-08,4.22,4.19,0.03
8,2017-09,4.15,4.14,0.01
9,2017-10,4.15,4.07,0.08


Excluding the lowest-rated sellers does not significantly impact the overall average review score, suggesting that other factors may be contributing to the negative customer feedback.

Based on the patterns identified in the worst-performing seller, we’ll now analyze delivery times to better understand their influence on review scores.

## Delivery Time by Year and Month

In [21]:
order =""" SELECT
    
    purchase_date
    ,ROUND(AVG(delivery_days),0) avg_delivery_days
    ,ROUND(AVG(estimated_delivery_days),0) avg_estimated_delivery_days
    ,ROUND((SUM(late_delivery) * 1.0 / COUNT(order_id))*100, 2) AS pct_late_deliveries
    ,ROUND(AVG(approvation_days),0) avg_approvation_days
    ,ROUND(AVG(aprovation_to_carrier_days),0) avg_aprovation_to_carrier_days
    ,ROUND(AVG(carrier_to_delivery_days),0) avg_carrier_to_delivery_days
            
        FROM 
            (SELECT 

            order_id
            ,strftime('%Y-%m', order_purchase_timestamp) purchase_date 
            ,julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) AS delivery_days
            ,julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) AS estimated_delivery_days
            ,CASE
                WHEN julianday(order_delivered_customer_date) > julianday(order_estimated_delivery_date) THEN 1
                ELSE 0
            END AS late_delivery
            ,julianday(order_approved_at) - julianday(order_purchase_timestamp) AS approvation_days
            ,julianday(order_delivered_carrier_date) - julianday(order_approved_at) AS aprovation_to_carrier_days
            ,julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date) AS carrier_to_delivery_days

            FROM orders

            WHERE order_status = 'delivered'
                AND strftime('%Y', order_purchase_timestamp) <> "2016") transformed_data_columns

        GROUP BY 1
        
        ;"""

pd.read_sql_query(order, conn)

,purchase_date,avg_delivery_days,avg_estimated_delivery_days,pct_late_deliveries,avg_approvation_days,avg_aprovation_to_carrier_days,avg_carrier_to_delivery_days
0,2017-01,13.0,40.0,3.07,0.0,3.0,9.0
1,2017-02,13.0,32.0,3.21,0.0,3.0,10.0
2,2017-03,13.0,25.0,5.58,0.0,3.0,10.0
3,2017-04,15.0,27.0,7.86,1.0,3.0,11.0
4,2017-05,11.0,24.0,3.61,0.0,3.0,8.0
5,2017-06,12.0,24.0,3.86,0.0,3.0,9.0
6,2017-07,12.0,23.0,3.43,0.0,3.0,8.0
7,2017-08,11.0,23.0,3.32,0.0,3.0,8.0
8,2017-09,12.0,22.0,5.20,0.0,3.0,9.0
9,2017-10,12.0,23.0,5.29,0.0,3.0,8.0


In November 2017, and in February and March 2018, there was a significant increase in the percentage of delayed deliveries.  
While the average delay rate typically ranged between 3% and 6%, these months saw peaks of 14%, 15%, and 21%, respectively.

This suggests that months with a high percentage of delayed deliveries tend to impact customer review scores in the following month.  
This one-month gap is explained by the average delivery time of 17 days — reviews are recorded upon delivery, not at the time of purchase.  
As a result, negative customer feedback tends to appear in the month after a delay-heavy sales period.

By breaking down the delivery process into stages, we can see that during months with high delay rates, the longest increase occurs between the moment the logistics partner receives the order and the final delivery to the customer.

This indicates a need for direct communication with the logistics providers responsible for these delays, in order to develop targeted corrective actions.

## Delivery Time by Seller

In [22]:
order =""" SELECT
    
    purchase_date
    ,ROUND(AVG(delivery_days),0) avg_delivery_days
    ,ROUND(AVG(estimated_delivery_days),0) avg_estimated_delivery_days
    ,ROUND((SUM(late_delivery) * 1.0 / COUNT(order_id))*100, 2) AS pct_late_deliveries
    ,ROUND(AVG(approvation_days),0) avg_approvation_days
    ,ROUND(AVG(aprovation_to_carrier_days),0) avg_aprovation_to_carrier_days
    ,ROUND(AVG(carrier_to_delivery_days),0) avg_carrier_to_delivery_days
            
        FROM 
            (SELECT 

            O.order_id
            ,strftime('%Y-%m', order_purchase_timestamp) purchase_date 
            ,julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) AS delivery_days
            ,julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) AS estimated_delivery_days
            ,CASE
                WHEN julianday(order_delivered_customer_date) > julianday(order_estimated_delivery_date) THEN 1
                ELSE 0
            END AS late_delivery
            ,julianday(order_approved_at) - julianday(order_purchase_timestamp) AS approvation_days
            ,julianday(order_delivered_carrier_date) - julianday(order_approved_at) AS aprovation_to_carrier_days
            ,julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date) AS carrier_to_delivery_days

            FROM orders O
            LEFT JOIN order_items OI ON OI.order_id = O.order_id
            LEFT JOIN sellers S ON OI.seller_id = S.seller_id 

            WHERE order_status = 'delivered'
                AND strftime('%Y', order_purchase_timestamp) <> "2016"
                AND S.seller_id NOT IN ('7c67e1448b00f6e969d365cea6b010ab','897060da8b9a21f655304d50fd935913','88460e8ebdecbfecb5f9601833981930','1835b56ce799e6a4dc4eddc053f04066','4c2b230173bb36f9b240f2b8ac11786e','1025f0e2d44d7041d6cf58b6550e0bfa','4a3ca9315b744ce9f8e9374361493884','d2374cbcbb3ca4ab1086534108cc3ab7','8160255418d5aaa7dbdc9f4c64ebda44','e5a3438891c0bfdb9394643f95273d8e')
            )
        GROUP BY 1
        
        """

pd.read_sql_query(order, conn)

,purchase_date,avg_delivery_days,avg_estimated_delivery_days,pct_late_deliveries,avg_approvation_days,avg_aprovation_to_carrier_days,avg_carrier_to_delivery_days
0,2017-01,13.0,40.0,2.96,0.0,3.0,9.0
1,2017-02,13.0,32.0,3.40,0.0,3.0,9.0
2,2017-03,13.0,25.0,5.43,0.0,3.0,10.0
3,2017-04,15.0,27.0,7.67,1.0,3.0,11.0
4,2017-05,11.0,24.0,3.72,0.0,3.0,8.0
5,2017-06,12.0,24.0,3.87,0.0,3.0,9.0
6,2017-07,11.0,23.0,3.67,0.0,3.0,8.0
7,2017-08,11.0,24.0,3.10,0.0,3.0,8.0
8,2017-09,12.0,22.0,5.10,0.0,3.0,9.0
9,2017-10,12.0,23.0,5.05,0.0,3.0,8.0


The delivery delays are not primarily caused by low-performing sellers, but likely stem from other factors.

Even after filtering out the worst-rated sellers, the average delivery time remains elevated during the problematic months compared to historical trends.

Next, we’ll analyze delivery performance across different geographic regions.

## Delivery Time by State

In [23]:
order =""" SELECT
    
    customer_state
    ,ROUND(AVG(delivery_days),0) avg_delivery_days
    ,ROUND(AVG(estimated_delivery_days),0) avg_estimated_delivery_days
    ,ROUND((SUM(late_delivery) * 1.0 / COUNT(order_id))*100, 2) AS pct_late_deliveries
    ,ROUND(AVG(approvation_days),0) avg_approvation_days
    ,ROUND(AVG(aprovation_to_carrier_days),0) avg_aprovation_to_carrier_days
    ,ROUND(AVG(carrier_to_delivery_days),0) avg_carrier_to_delivery_days
            
        FROM 
            (SELECT 

            O.order_id
            ,C.customer_state
            ,strftime('%Y-%m', order_purchase_timestamp) purchase_date 
            ,julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) AS delivery_days
            ,julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) AS estimated_delivery_days
            ,CASE
                WHEN julianday(order_delivered_customer_date) > julianday(order_estimated_delivery_date) THEN 1
                ELSE 0
            END AS late_delivery
            ,julianday(order_approved_at) - julianday(order_purchase_timestamp) AS approvation_days
            ,julianday(order_delivered_carrier_date) - julianday(order_approved_at) AS aprovation_to_carrier_days
            ,julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date) AS carrier_to_delivery_days

            FROM orders O
            LEFT JOIN customers C ON C.customer_id = O.customer_id 

            WHERE order_status = 'delivered'
                AND strftime('%Y', order_purchase_timestamp) <> "2016"
                            )
        GROUP BY 1
        ORDER BY 4 DESC
        
        """

pd.read_sql_query(order, conn)

,customer_state,avg_delivery_days,avg_estimated_delivery_days,pct_late_deliveries,avg_approvation_days,avg_aprovation_to_carrier_days,avg_carrier_to_delivery_days
0,AL,25.0,32.0,23.99,0.0,3.0,21.0
1,MA,22.0,30.0,19.64,1.0,3.0,18.0
2,PI,19.0,30.0,16.00,0.0,3.0,16.0
3,CE,21.0,31.0,15.40,0.0,3.0,18.0
4,SE,21.0,31.0,15.36,0.0,3.0,18.0
5,BA,19.0,29.0,14.05,0.0,3.0,16.0
6,RJ,15.0,26.0,13.52,0.0,3.0,12.0
7,TO,18.0,29.0,12.77,1.0,3.0,14.0
8,RR,30.0,45.0,12.50,0.0,3.0,26.0
9,PA,24.0,37.0,12.42,1.0,3.0,20.0


Several states exhibit high delivery delay rates:

- 15 states recorded delay percentages above 10% in 2017 and 2018.  
- Among them, 5 states had delay rates exceeding 15%.

In [24]:
order =""" SELECT
    
    customer_state
    ,ROUND(AVG(delivery_days),0) avg_delivery_days
    ,ROUND(AVG(estimated_delivery_days),0) avg_estimated_delivery_days
    ,ROUND((SUM(late_delivery) * 1.0 / COUNT(order_id))*100, 2) AS pct_late_deliveries
    ,ROUND(AVG(approvation_days),0) avg_approvation_days
    ,ROUND(AVG(aprovation_to_carrier_days),0) avg_aprovation_to_carrier_days
    ,ROUND(AVG(carrier_to_delivery_days),0) avg_carrier_to_delivery_days
            
        FROM 
            (SELECT 

            O.order_id
            ,C.customer_state
            ,strftime('%Y-%m', order_purchase_timestamp) purchase_date 
            ,julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp) AS delivery_days
            ,julianday(order_estimated_delivery_date) - julianday(order_purchase_timestamp) AS estimated_delivery_days
            ,CASE
                WHEN julianday(order_delivered_customer_date) > julianday(order_estimated_delivery_date) THEN 1
                ELSE 0
            END AS late_delivery
            ,julianday(order_approved_at) - julianday(order_purchase_timestamp) AS approvation_days
            ,julianday(order_delivered_carrier_date) - julianday(order_approved_at) AS aprovation_to_carrier_days
            ,julianday(order_delivered_customer_date) - julianday(order_delivered_carrier_date) AS carrier_to_delivery_days

            FROM orders O
            LEFT JOIN customers C ON C.customer_id = O.customer_id 

            WHERE order_status = 'delivered'
                AND strftime('%Y', order_purchase_timestamp) <> "2016"
                AND strftime('%Y-%M', order_purchase_timestamp) IN ('2018-03','2018-02','2017-11')
                            )
        GROUP BY 1
        ORDER BY 4 DESC
        
        """

pd.read_sql_query(order, conn)

,customer_state,avg_delivery_days,avg_estimated_delivery_days,pct_late_deliveries,avg_approvation_days,avg_aprovation_to_carrier_days,avg_carrier_to_delivery_days
0,CE,24.0,31.0,25.81,0.0,2.0,21.0
1,SE,22.0,34.0,25.00,0.0,7.0,15.0
2,BA,20.0,29.0,18.18,0.0,3.0,17.0
3,ES,18.0,25.0,15.22,1.0,3.0,14.0
4,MA,21.0,29.0,15.00,0.0,2.0,18.0
5,RJ,15.0,26.0,14.58,0.0,3.0,12.0
6,MT,24.0,34.0,14.29,1.0,2.0,22.0
7,SC,15.0,26.0,10.23,0.0,3.0,12.0
8,PB,25.0,38.0,10.00,0.0,3.0,21.0
9,DF,14.0,23.0,9.26,0.0,3.0,11.0


In the three worst months for customer reviews, three of the five states with the lowest historical review scores also appeared.

- In the top five, Ceará (CE) and Sergipe (SE) showed the most significant increases in delivery delays — 10.4% and 9.6% above the historical average, respectively.  
- Maranhão (MA) also appeared in the top five, despite having a 4.6% decrease in its delay rate compared to the historical average.  
- The other two states in the top five — Bahia (BA) and Espírito Santo (ES) — had increases between 3% and 4%.  
- In SE, there was a spike in the time between order approval and delivery to the logistics partner: 7 days vs. the historical average of 3 days.  
- Mato Grosso (MT) recorded an 8% increase in delivery delays.

On the other hand, Piauí (PI) and Alagoas (AL) had 0% delivery delays during this same period — indicating that although they are historically problematic states for delivery, they were not responsible for the KPI decline during these specific months.

# Conclusions

There was a revenue increase of **21,04%** in 2018 compared to 2017 (Jan–Aug 2018 vs. full-year 2017).

This growth was primarily driven by:
- The state of São Paulo, which grew by **33%**, and  
- Key product categories such as **Health and Beauty** (`beleza_saude`), **Gifts and Watches** (`relogios_presentes`), and **Home Utilities** (`utilidades_domesticas`).

The data suggests that a **sales strategy focused on quantity** is more effective in this marketplace than one focused on high prices.

- Among the top 15 highest-revenue categories, 13 also appeared in the top 15 for quantity sold, while only 2 were in the top 15 for average price.

In other words, revenue leaders in the sales mix are mostly high-volume products — with only one high-priced product category appearing among top earners.

Additionally, **12% of total revenue in 2018 came from 37 new sellers**, reinforcing the importance of onboarding and attracting new partners to the marketplace.

Finally, most negative customer reviews are tied to **delivery delays**.  
States with the worst review scores often overlap with those experiencing the highest delay rates — confirming that delivery performance plays a crucial role in customer satisfaction.
